# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/Benchmarks/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data_set = 'UNSW-NB15'  #@param {type: "string"}
df = pd.read_csv(root_path+data_set+'.csv', header=None)

#cleaned data without non-attack values
# X = pd.read_csv(root_path+data_set+'.csv', header=None, usecols=[i for i in range(48)])
# Y = pd.read_csv(root_path+data_set+'.csv', header=None, usecols=[48])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Cleaning

In [0]:
df.columns=['srcip','sport','dstip','dsport',

'proto',
'state',
'dur',
'sbytes',
'dbytes',
'sttl',
'dttl',
'sloss',
'dloss',
'service',
'Sload',
'Dload',
'Spkts',
'Dpkts',
'swin',
'dwin',
'stcpb',
'dtcpb',
'smeansz',
'dmeansz',
'trans_depth',
'res_bdy_len',
'Sjit',
'Djit',
'Stime',
'Ltime',
'Sintpkt',
'Dintpkt',
'cprtt',
'synack',
'ackdat',
'is_sm_ips_ports',
'ct_state_ttl',
'ct_flw_http_mthd',
'is_ftp_login',
'ct_ftp_cmd',
'ct_srv_src',
'ct_srv_dst',
'ct_dst_ltm',
'ct_src_ ltm',
'ct_src_dport_ltm',
'ct_dst_sport_ltm',
'ct_dst_src_ltm',
'attack_cat',
'Label']

In [0]:
df=df.drop(df.columns[:4],axis=1)

In [0]:
df = df.drop(columns=['attack_cat'])

In [0]:
df = df.dropna()

In [0]:
df.ct_ftp_cmd.unique()

array([0, 1, 6, 2, 4, 8, 5, 3, '0', '1'], dtype=object)

In [0]:
df['ct_ftp_cmd']=df['ct_ftp_cmd'].replace([0, 1, 6, 2, 4, 8, 5, 3, '0', '1', ' ', '2', '4'],[0,1,6,2,4,8,5,3,0,1,0,2,4])

In [0]:
df['state'].unique()

array(['CON', 'INT', 'FIN', 'URH', 'REQ', 'ECO', 'RST', 'CLO', 'TXD',
       'URN', 'no', 'ACC', 'PAR', 'MAS', 'TST', 'ECR'], dtype=object)

In [0]:
df['proto'].unique()

array(['udp', 'arp', 'tcp', 'ospf', 'icmp', 'igmp', 'sctp', 'udt', 'sep',
       'sun-nd', 'swipe', 'mobile', 'pim', 'rtp', 'ipnip', 'ip', 'ggp',
       'st2', 'egp', 'cbt', 'emcon', 'nvp', 'igp', 'xnet', 'argus',
       'bbn-rcc', 'chaos', 'pup', 'hmp', 'mux', 'dcn', 'prm', 'trunk-1',
       'xns-idp', 'trunk-2', 'leaf-1', 'leaf-2', 'irtp', 'rdp', 'iso-tp4',
       'netblt', 'mfe-nsp', 'merit-inp', '3pc', 'xtp', 'idpr', 'tp++',
       'ddp', 'idpr-cmtp', 'ipv6', 'il', 'idrp', 'ipv6-frag', 'sdrp',
       'ipv6-route', 'gre', 'rsvp', 'mhrp', 'bna', 'esp', 'i-nlsp',
       'narp', 'ipv6-no', 'tlsp', 'skip', 'ipv6-opts', 'any', 'cftp',
       'sat-expak', 'kryptolan', 'rvd', 'ippc', 'sat-mon', 'ipcv', 'visa',
       'cpnx', 'cphb', 'wsn', 'pvp', 'br-sat-mon', 'wb-mon', 'wb-expak',
       'iso-ip', 'secure-vmtp', 'vmtp', 'vines', 'ttp', 'nsfnet-igp',
       'dgp', 'tcf', 'eigrp', 'sprite-rpc', 'larp', 'mtp', 'ax.25',
       'ipip', 'micp', 'aes-sp3-d', 'encap', 'etherip', 'pri-enc', 'gmtp'

In [0]:
l1=list(np.arange(len(df['state'].unique())))
df['state']=df['state'].replace(['CON', 'INT', 'FIN', 'URH', 'REQ', 'ECO', 'RST', 'CLO', 'TXD',
       'URN', 'no', 'ACC', 'PAR', 'MAS', 'TST', 'ECR'],l1)

In [0]:
l1=list(np.arange(len(df['service'].unique())))
df['service']=df['service'].replace(['-', 'http', 'smtp', 'pop3', 'snmp', 'ftp', 'ftp-data', 'ssl',
       'irc', 'dns', 'radius', 'dhcp', 'ssh'],l1)

In [0]:
l1=list(np.arange(len(df['proto'].unique())))
df['proto']=df['proto'].replace(['udp', 'arp', 'tcp', 'ospf', 'icmp', 'igmp', 'sctp', 'udt', 'sep',
       'sun-nd', 'swipe', 'mobile', 'pim', 'rtp', 'ipnip', 'ip', 'ggp',
       'st2', 'egp', 'cbt', 'emcon', 'nvp', 'igp', 'xnet', 'argus',
       'bbn-rcc', 'chaos', 'pup', 'hmp', 'mux', 'dcn', 'prm', 'trunk-1',
       'xns-idp', 'trunk-2', 'leaf-1', 'leaf-2', 'irtp', 'rdp', 'iso-tp4',
       'netblt', 'mfe-nsp', 'merit-inp', '3pc', 'xtp', 'idpr', 'tp++',
       'ddp', 'idpr-cmtp', 'ipv6', 'il', 'idrp', 'ipv6-frag', 'sdrp',
       'ipv6-route', 'gre', 'rsvp', 'mhrp', 'bna', 'esp', 'i-nlsp',
       'narp', 'ipv6-no', 'tlsp', 'skip', 'ipv6-opts', 'any', 'cftp',
       'sat-expak', 'kryptolan', 'rvd', 'ippc', 'sat-mon', 'ipcv', 'visa',
       'cpnx', 'cphb', 'wsn', 'pvp', 'br-sat-mon', 'wb-mon', 'wb-expak',
       'iso-ip', 'secure-vmtp', 'vmtp', 'vines', 'ttp', 'nsfnet-igp',
       'dgp', 'tcf', 'eigrp', 'sprite-rpc', 'larp', 'mtp', 'ax.25',
       'ipip', 'micp', 'aes-sp3-d', 'encap', 'etherip', 'pri-enc', 'gmtp',
       'pnni', 'ifmp', 'aris', 'qnx', 'a/n', 'scps', 'snp', 'ipcomp',
       'compaq-peer', 'ipx-n-ip', 'vrrp', 'zero', 'pgm', 'iatp', 'ddx',
       'l2tp', 'srp', 'stp', 'smp', 'uti', 'sm', 'ptp', 'fire', 'crtp',
       'isis', 'crudp', 'sccopmce', 'sps', 'pipe', 'iplt', 'unas', 'fc',
       'ib'],l1)

In [0]:
X=df.iloc[:,:-1]
Y=df.iloc[:,-1:]

In [0]:
from sklearn.preprocessing import StandardScaler
X=X.astype('float')
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

# Feature Engineering

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)


# Rank the features and sort
x = list(zip(X.columns, clf.feature_importances_))
f = []
for i in range(len(X.columns)):
  if (x[i][1] >= .05):
    f.append(x[i][0])


X = X[f]
print(clf.feature_importances_)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[2.03138655e-02 2.11887217e-03 2.67839547e-02 3.50784682e-02
 6.90436217e-03 1.28813911e-01 5.63891906e-02 3.53267841e-03
 1.33868128e-03 4.11448386e-03 3.90648668e-02 1.93014273e-02
 1.01028458e-02 1.78442081e-03 3.52449267e-03 4.91251600e-05
 3.35530824e-03 2.27076452e-03 3.58366083e-02 2.43440802e-03
 1.72381373e-04 0.00000000e+00 4.87740996e-03 6.52298885e-03
 1.85059484e-02 2.43655320e-02 1.43429325e-02 3.05646417e-02
 2.89020731e-02 7.45817634e-03 3.24183070e-02 1.58900264e-06
 3.76722645e-01 4.71938889e-05 0.00000000e+00 0.00000000e+00
 9.85090047e-03 2.07248439e-02 4.60009107e-03 4.71459799e-03
 3.67529182e-03 3.27800281e-03 5.14171711e-03]


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

In [0]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:

x = list(zip(X.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X.columns)):
    print(x[i])
    if (i==len(X.columns)-1):
      print("features",i+1)

(5, 0.4387182800584643)
(32, 0.3805453042572515)
(6, 0.18073641568428414)
features 3


In [0]:
print(clf.score(x_train, y_train), "train")

0.9963886672446025 train


In [0]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

              precision    recall  f1-score   support

           0     0.9998    0.9965    0.9981    212957
           1     0.8554    0.9895    0.9176      4484

    accuracy                         0.9963    217441
   macro avg     0.9276    0.9930    0.9579    217441
weighted avg     0.9968    0.9963    0.9965    217441

